# EDA avion

**Context:** Dans le but de mettre à disposition une interface permettant de prédire si un vol est en retard, nous avons analyser une base de données comportant differentes données enregistré sur les vols de compagnies aeriennes en 2016.

# Import

In [ ]:
# import des librairie
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pymysql
import fastparquet
from function import *
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Lien avec la base de données

Nous avons crée une base de donnée contenant 12 tables, une par mois de l'années 2016.

In [ ]:
conn=pymysql.connect(host='localhost',port=int(3306),user='root',passwd='groot',db='retard_avion')

In [ ]:
data=pd.read_sql_query("SELECT * FROM janvier " ,conn)
customers

In [ ]:
# voir si le dataframe contient des NaN
count_nan(data)

In [ ]:
# enlevé les colonnes qui possède plus de 70% de NaN
data = delete_nan(data, 70)

Nous n'avons pas observé de lignes dupliqué dans le dataframe. 
Nous avons rencontré des difficulté a pouvoir analyser la totalité des données nous avons selectionné les colonnes qui nous parraissait le plus pertinante et nous avons transformé le dataframe en fichier parquet

In [ ]:
df = pd.read_parquet('df.parquet' , engine='fastparquet')

In [ ]:
duplication(df)

# valeurs abérentes

In [ ]:
categ = df.select_dtypes(include=['object']).columns
num = df._get_numeric_data().columns

In [ ]:
numerical = df[num]
numerical.describe()

In [ ]:
catego = df[categ]
for i in catego.columns :
    print("Pour",i,":")
    print("-----------")
    print(catego[i].value_counts())
    print("-----------")

# outlier

In [ ]:
boxplots(df, num)

In [ ]:
distance = outlier_identificator(df,'DISTANCE')
distance

# data viz

## univarié

In [ ]:
univar_repartition(df, num)

In [ ]:
univar_repartition(df, categ)

## multivarié

In [ ]:
histogram_plotly( df, 'CARRIER',hue ='ARR_DEL15')

In [ ]:
histogram_plotly(df,'MONTH',hue ='ARR_DEL15')

In [ ]:
histogram_plotly(df,'DAY_OF_WEEK',hue ='ARR_DEL15')

In [ ]:
histogram_plotly(df,'ORIGIN_STATE_NM',hue ='ARR_DEL15')

In [ ]:
histogram_plotly(df,'DEST_STATE_NM',hue ='ARR_DEL15')

In [ ]:
histogram_plotly(df,'DEP_DELAY',hue = 'ARR_DEL15')

In [ ]:
# calculer le nombre de vols en retard
delayed = df[df["ARR_DEL15"] == 1]
ontime = df[df["ARR_DEL15"] == 0]
num_delayed = len(delayed)
num_ontime = len(ontime)

# créer le pie chart
labels = ["DELAYED", "ONTIME"]
sizes = [num_delayed, num_ontime]
colors = [ "#7FFFD4" ,"#6CDFDF"]

fig, ax = plt.subplots()
ax.pie(sizes, colors=colors, labels=labels, autopct='%1.1f%%', startangle=90)
plt.title("Delayed Flights")
plt.show()

In [ ]:
# tracer le scatter plot
sns.scatterplot(x="DISTANCE", y="DEP_DELAY", hue="ARR_DEL15", data=df, palette=[ "#7FFFD4" ,"#6CDFDF"])
plt.title("Temps de retard du vol par rapport à la distance du vol")
plt.ylabel("Retard ( en min )")
plt.show()

In [ ]:
# Créer le plot avec Seaborn
sns.lineplot(x='MONTH', y='DISTANCE', data=df, color='#7FFFD4')

# Ajouter un titre
plt.title("Distance moyenne d'un vol pour chaque mois")

# Afficher le plot
plt.show()

In [ ]:
# Créer le plot avec Seaborn
sns.lineplot(x='MONTH', y='DEP_DELAY', data=df, color='#7FFFD4', ci = None)

# Ajouter un titre
plt.title("Retard moyen mensuel")

# Afficher le plot
plt.show()

Pour avoir une meilleur idée de l'impacte du retard, nous avons calculé le pourcentage de retard

In [ ]:
compagnie= pourcent(df,["CARRIER"],'ARR_DEL15')
pourcent_fig(compagnie,"CARRIER" )

In [ ]:
month = pourcent(df, ['MONTH'],'ARR_DEL15')
pourcent_fig(month,'MONTH' )

In [ ]:
dayw= pourcent(df, ['DAY_OF_WEEK'],'ARR_DEL15')
pourcent_fig(dayw,'DAY_OF_WEEK' )

In [ ]:
daym= pourcent(df,['DAY_OF_MONTH'],'ARR_DEL15')
pourcent_fig(daym,'DAY_OF_MONTH' )

In [ ]:
dis= pourcent(df,['DISTANCE'],'ARR_DEL15')
pourcent_fig(dis,'DISTANCE' )

In [ ]:
ori= pourcent(df,['ORIGIN_STATE_NM'],'ARR_DEL15')
pourcent_fig(ori,'ORIGIN_STATE_NM' )

In [ ]:
dest= pourcent(df,['DEST_STATE_NM'],'ARR_DEL15')
pourcent_fig(dest,'DEST_STATE_NM' )

# corrélation et colinéarité

## corrélation

In [ ]:
plt.figure(figsize=(15, 10))
sns.heatmap(numerical.corr(), annot=True)

# colinéarité

Nous avons essayé de tester la multicolinearité des colonnes. 

In [ ]:
X = numerical.copy() 
multicollinearity_check(X, thresh=5.0) 
# VIF (Variance Inflation Factor) > 10 as an indicator of multicollinearity  https://quantifyinghealth.com/vif-threshold/